In [10]:
import getpass
import paramiko
import os
import tarfile
import wradlib as wrl
import cv2

In [11]:
#Decides if files should be downloaded and extracted
download = False
extractFiles = False

Funktionen zum downloaden der Daten

In [12]:
def printProgress(transferred, toBeTransferred):

    width = 50
    percent = transferred / toBeTransferred
    size = int(percent * width)
    fetched = transferred / (1024**2)
    total = toBeTransferred / (1024**2)
    progress = "%(current)d/%(total)d Mb"

    args = {
        "current": fetched,
        "total": total
    }
    bar = progress + ' [' + '#' * size + '#' + '.'*(width - size) + ']'
    print("\r"+bar % args, end='')



def downloadDataSet(root_dir,host,port,DOWNLOAD_PATH):

    path = root_dir

    transport = paramiko.Transport((host, port))

    print("Downloading from:", str(host + "/"+DOWNLOAD_PATH))
    username = input('Enter your username\n')
    pswd = getpass.getpass('Enter your password:\n')
    print("Connecting to "+host+"...")

    transport.connect(username=username, password=pswd)
    sftp = paramiko.SFTPClient.from_transport(transport)

    if os.path.exists(path) == False:
        os.makedirs(path)

    print("Downloading...")
    sftp.get(remotepath=DOWNLOAD_PATH, localpath=path +
                 "/"+DOWNLOAD_PATH.split("/")[-1], callback=printProgress)
    print("\n")

Funktionen zum transformieren der Daten zu PNGs

In [46]:
def initTranformRadarToPng ():
    RadarValues99 = np.asanyarray(pickle.load(open("RadarValues99PercentDatapoints.p", "rb")))
    maxValue = 40 # actual maxVal = maxVal + 99% Quanil (which is 1,39)
    remaining_range = 255-len(RadarValues99)
    scaleFactor = remaining_range/maxValue
    maxOfRadarValues99 = max(RadarValues99[:,0])
    RadarValues99 = np.sort(RadarValues99[:,0])
    return RadarValues99, maxOfRadarValues99, scaleFactor
    
def transformRadarDataToPng (radarData, RadarValues99, maxOfRadarValues99, scaleFactor):
    #scale values which are larger than maxOfRadarValues99
    radarData = np.where(radarData>maxOfRadarValues99, 
                         ((radarData-maxOfRadarValues99)*scaleFactor+len(RadarValues99)).astype(int), 
                         radarData)
    #set values that are larger than 255 to 255
    radarData[radarData>255]=255

    for idx in range(1,len(RadarValues99)):
        radarData[(radarData<=RadarValues99[-idx]) & (radarData>RadarValues99[-idx-1])] = len(RadarValues99)-idx

    radarData[(radarData>0) & (radarData<RadarValues99[0])]=1
    return radarData

# Array-Like, max of all data
def normalize(data, absolute_max):
    factor = float(254)/absolute_max
    data_scaled = []
    for row in data:
        scaled_row = []
        for pixel in row:
            if pixel == 0:
                scaled_row.append(pixel)
            else:
                scaled_row.append(pixel*factor+1)
        data_scaled.append(scaled_row)

    return np.asanyarray(data_scaled)

def save_png_grayscale_8bit(image_data, filepath, filename, factor=1):
    image_data_8bit = image_data.astype(np.uint8)
    image_data_8bit *= int(factor)
    
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    full_filename = filepath + filename + ".png"
    cv2.imwrite(full_filename, image_data_8bit)
    
def get_timestamp_for_bin_filename(bin_file_name):
    split = bin_file_name.split('-')
    timestamp = split[2]
    return timestamp

In [47]:
HOST = "iosds01.ios.htwg-konstanz.de"
PORT = 22

#Muss nachher dynamisch bestimmt werden
#Datei die gedownloadet werden soll
ZIP_TO_DOWNLOAD = "/YW2017.002_200801.tar"

#Speicherort der Datei die gedownloadet werden soll
DOWNLOAD_PATH = "./datascience/regenradar/Radardaten/2008" + ZIP_TO_DOWNLOAD

#Ort an dem alle Daten gespeichert werden sollen ( in Unterordnern)
STORE_PATH = "/home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data"

#Ort an dem alle Radardaten von einem Monat gezippt vom Server gespeichert werden
PATH_TO_RADAR_ZIP = STORE_PATH + "/RadDataZIP"

#Download data from Server if download is True
if(download == True):
    downloadDataSet("./Data/RadDataZIP",HOST,PORT,DOWNLOAD_PATH)

In [48]:
#Extract the downloaded data, if extractFiles is True
if extractFiles == True:
    tar_year = tarfile.open(PATH_TO_RADAR_ZIP + ZIP_TO_DOWNLOAD, "r:")
    tar_year.extractall(STORE_PATH + "/RadDataZipMonth")
    list_month = os.listdir(STORE_PATH + "/RadDataZipMonth")
    for month in list_month:
        tar_month = tarfile.open(STORE_PATH + "/RadDataZipMonth/" + month, "r:gz")
        tar_month.extractall(STORE_PATH + "/MonthsRadarData")


In [55]:
#names of all bin Data in 5 min. resolution are listed 
listOfRadarBin5MinSteps = os.listdir(STORE_PATH + "/MonthsRadarData")

#counter to log the progress
counter = 0

for binData in listOfRadarBin5MinSteps: 
    data, attrs = wrl.io.read_radolan_composite(STORE_PATH + "/MonthsRadarData/" + binData)
    data = normalize(data, 5)
    
    #Store the Data in /home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/<NameOfDownloadedZip>
    save_png_grayscale_8bit(data, STORE_PATH + "/MonthPNGData" + ZIP_TO_DOWNLOAD[:-4] + "/", get_timestamp_for_bin_filename(binData), factor=1)
    
    counter = counter + 1
    print("Picture {0} / {1}".format(counter, len(listOfRadarBin5MinSteps)), end="\r")

/home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/YW2017.002_200801/0801191440.png
/home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/YW2017.002_200801/0801181340.png
/home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/YW2017.002_200801/0801150825.png
/home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/YW2017.002_200801/0801020110.png


KeyboardInterrupt: 

In [ ]:
#Hier wieder alle binären und Zipdaten löschen